In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=User.name,
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
import warnings


with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase()
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
db[User].values()


[
    User({'name': 'John', 'age': 31, '_id': 'e464e307-1e01-4c53-8b3c-cfbd7f66a1b4'}),
    User({'name': 'Jane', 'age': 56, '_id': '0abb9c96-b6ca-4150-87df-8bd7c4b66595'}),
    User({'name': 'Jack', 'age': 66, '_id': 'df1594ae-3658-419d-90a1-82822d1f0977'})
]

In [9]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results].items())


[
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
db[Search.results].to_df()

term,test_db_schema_Project_number,number,status,name,test_db_schema_Organization__id,start,end
str,i64,i64,str,str,str,date,date
"""test""",1,1,"""done""","""cleaning shoes""","""615c7cd6-9e45-4637-80e7-48f2cb…",2020-01-02,2020-01-05
"""test""",0,0,"""done""","""baking cake""","""cdc91022-0920-413c-b1fc-4e8105…",2020-01-01,2020-01-04
"""test""",2,2,"""started""","""fixing cars""","""403e122e-1063-49aa-b51e-435943…",2020-01-03,2020-01-06


In [14]:
db[Organization].to_df()

_id,city,address,name
str,str,str,str
"""615c7cd6-9e45-4637-80e7-48f2cb…","""Bakerville""","""Main Street 1""","""Bakery"""
"""cdc91022-0920-413c-b1fc-4e8105…","""Shoetown""","""Main Street 2""","""Shoe Shop"""
"""403e122e-1063-49aa-b51e-435943…","""Cartown""","""Main Street 3""","""Car Shop"""


In [15]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         541574 function calls (507694 primitive calls) in 1.147 seconds



   Ordered by: cumulative time
   List reduced from 2270 to 143 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.001    0.000    0.527    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:690(_load_rec_from_item)
       52    0.002    0.000    0.523    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:569(_load_record)
       23    0.000    0.000    0.416    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1368(__ior__)
       23    0.001    0.000    0.415    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2794(_mutate)
       56    0.000    0.000    0.378    0.007 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:831(load)
   168/57    0.001    0.000    0.348    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:658(_load_records)
16924/8403    0.029    0.000    0.300    0.000 C:\User